In [ ]:
import yfinance as yf
from datetime import datetime
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots


In [36]:
data = yf.download("RELIANCE.NS", start="2020-01-01", end=datetime.now(), interval="1d", rounding=True)
data.columns = [col[0] for col in data.columns]


[*********************100%***********************]  1 of 1 completed


In [37]:
# Plot the candlestick chart
# mpf.plot(data, type='candle', style='charles', volume=False, mav = (20,50), figsize=(14, 8), title=f"Candlestick Chart for {ticker_symbol}")

In [64]:
# Step 1: Add RSI
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = calculate_rsi(df['Close'])

# Step 2: Buy/Sell Conditions
df['Buy Signal'] = (df['RSI'].shift(1) < 30) & (df['RSI'] > 30)
df['Sell Signal'] = (df['RSI'].shift(1) > 70) & (df['RSI'] < 70)

# Step 3: Backtest with Trade Log
in_position = False
entry_price = 0
entry_date = None

trade_log = []

for i in range(1, len(df)):
    current_date = df.index[i]
    price = df['Close'].iloc[i]

    if not in_position:
        if df['Buy Signal'].iloc[i]:
            in_position = True
            entry_price = price
            entry_date = current_date
    else:
        stoploss_triggered = price <= entry_price * 0.98
        rsi_sell = df['Sell Signal'].iloc[i]

        if stoploss_triggered or rsi_sell:
            exit_price = price
            exit_date = current_date
            reason = 'Stoploss' if stoploss_triggered else 'RSI Exit'
            pnl = (exit_price - entry_price) / entry_price

            trade_log.append({
                'Entry Date': entry_date,
                'Entry Price': round(entry_price, 2),
                'Exit Date': exit_date,
                'Exit Price': round(exit_price, 2),
                'P&L %': round(pnl * 100, 2),
                'Exit Reason': reason
            })

            in_position = False

# Step 4: Show Trade Log
trade_df = pd.DataFrame(trade_log)
print(trade_df)

# Step 5: Optional Summary
total_return = (trade_df['P&L %'] / 100 + 1).prod()
print(f"\n📊 Final Strategy Return: {total_return:.2f}x")
print(f"✅ Total Trades: {len(trade_df)}")
print(f"📈 Win Rate: {(trade_df['P&L %'] > 0).mean() * 100:.2f}%")

                  Entry Date  Entry Price                 Exit Date  \
0  2025-01-14 07:45:00+00:00      1231.25 2025-01-21 03:45:00+00:00   
1  2025-01-21 09:45:00+00:00      1277.00 2025-01-24 07:45:00+00:00   
2  2025-01-28 05:45:00+00:00      1236.65 2025-01-30 04:45:00+00:00   
3  2025-02-07 08:45:00+00:00      1267.80 2025-02-11 06:45:00+00:00   
4  2025-02-12 06:45:00+00:00      1217.30 2025-02-18 08:45:00+00:00   
5  2025-02-27 04:45:00+00:00      1206.15 2025-03-03 03:45:00+00:00   
6  2025-03-03 09:45:00+00:00      1174.00 2025-03-10 08:45:00+00:00   
7  2025-03-17 09:45:00+00:00      1241.75 2025-03-25 03:45:00+00:00   
8  2025-03-27 03:45:00+00:00      1282.80 2025-04-01 07:45:00+00:00   
9  2025-04-03 04:45:00+00:00      1247.65 2025-04-04 03:45:00+00:00   
10 2025-04-08 03:45:00+00:00      1169.45 2025-04-09 09:45:00+00:00   

    Exit Price  P&L % Exit Reason  
0      1286.80   4.51    RSI Exit  
1      1250.55  -2.07    Stoploss  
2      1246.90   0.83    RSI Exit  
3  

In [ ]:
# Step 1: Create a figure with 2 rows (1 for price, 1 for RSI)
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=("RELIANCE Candlestick", "RSI Indicator")
)

# Step 2: Candlestick chart (Row 1)
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Candlestick'
), row=1, col=1)

# Step 3: RSI line (Row 2)
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['RSI'],
    mode='lines',
    name='RSI',
    line=dict(color='blue')
), row=2, col=1)

# Step 4: Add RSI 30 and 70 lines
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1, annotation_text="Overbought (70)")
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1, annotation_text="Oversold (30)")

# Step 5: Layout
fig.update_layout(
    title='RELIANCE - Candlestick with RSI (14)',
    xaxis_rangeslider_visible=False,
    height=800,
    showlegend=False
)

fig.show()

In [65]:
lot_size = 500  # 🧾 Lot size per trade

# Step 1: Add ₹ P&L to trade log
for trade in trade_log:
    entry = trade['Entry Price']
    exit_ = trade['Exit Price']
    pnl_per_share = exit_ - entry
    total_pnl = pnl_per_share * lot_size

    trade['P&L ₹'] = round(total_pnl, 2)

# Step 2: Create DataFrame
trade_df = pd.DataFrame(trade_log)

# Step 3: Show Trade Log
print(trade_df)

# Step 4: Summary
total_return = (trade_df['P&L %'] / 100 + 1).prod()
total_profit_rs = trade_df['P&L ₹'].sum()
win_rate = (trade_df['P&L %'] > 0).mean() * 100

print(f"\n📊 Final Strategy Return: {total_return:.2f}x")
print(f"✅ Total Trades: {len(trade_df)}")
print(f"📈 Win Rate: {win_rate:.2f}%")
print(f"💰 Total Profit/Loss: ₹{total_profit_rs:,.2f}")

                  Entry Date  Entry Price                 Exit Date  \
0  2025-01-14 07:45:00+00:00      1231.25 2025-01-21 03:45:00+00:00   
1  2025-01-21 09:45:00+00:00      1277.00 2025-01-24 07:45:00+00:00   
2  2025-01-28 05:45:00+00:00      1236.65 2025-01-30 04:45:00+00:00   
3  2025-02-07 08:45:00+00:00      1267.80 2025-02-11 06:45:00+00:00   
4  2025-02-12 06:45:00+00:00      1217.30 2025-02-18 08:45:00+00:00   
5  2025-02-27 04:45:00+00:00      1206.15 2025-03-03 03:45:00+00:00   
6  2025-03-03 09:45:00+00:00      1174.00 2025-03-10 08:45:00+00:00   
7  2025-03-17 09:45:00+00:00      1241.75 2025-03-25 03:45:00+00:00   
8  2025-03-27 03:45:00+00:00      1282.80 2025-04-01 07:45:00+00:00   
9  2025-04-03 04:45:00+00:00      1247.65 2025-04-04 03:45:00+00:00   
10 2025-04-08 03:45:00+00:00      1169.45 2025-04-09 09:45:00+00:00   

    Exit Price  P&L % Exit Reason    P&L ₹  
0      1286.80   4.51    RSI Exit  27775.0  
1      1250.55  -2.07    Stoploss -13225.0  
2      1246.

In [34]:
reliance_df_daily = pd.read_csv('/Users/himj/HDrive/AppDevStudio/TradingBot/data/processed/RELIANCE.NSE_D.csv', parse_dates=['Date'], index_col='Date')

In [ ]:
from plotly.subplots import make_subplots

# -- Assuming df already has 'Open', 'High', 'Low', 'Close', 'RSI' and Date as index

# Step 1: Create a figure with 2 rows (1 for price, 1 for RSI)
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=("RELIANCE Candlestick", "RSI Indicator")
)

# Step 2: Candlestick chart (Row 1)
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Candlestick'
), row=1, col=1)

# Step 3: RSI line (Row 2)
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['RSI'],
    mode='lines',
    name='RSI',
    line=dict(color='blue')
), row=2, col=1)

# Step 4: Add RSI 30 and 70 lines
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1, annotation_text="Overbought (70)")
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1, annotation_text="Oversold (30)")

# Step 5: Layout
fig.update_layout(
    title='RELIANCE - Candlestick with RSI (14)',
    xaxis_rangeslider_visible=False,
    height=800,
    showlegend=False
)

fig.show()